In [ ]:
import laspy
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Intrinsic Param

In [ ]:
# Intrinsic Param
fx = 0
fy = 0
k1 = 0
k2 = 0
k3 = 0
k4 = 0
Cx = 0
Cy = 0
P1 = 0
P2 = 0
Nx = 0
Ny = 0
dx = 0
dy = 0


### Extrinsic Param

In [ ]:
# Extrinsic Param
filename = "test.jpg"
roll = 0
pitch = 0
yaw = 0

omega = 0
phi = 0
kappa = 0

camera_center = (0, 0, 0)
direction = (0, 0, 0)
up = (0, 0, 0)


### Read Point CLoud

In [ ]:
def read_laz_file(file_path: str) -> np.ndarray:
    point_cloud = laspy.read(file_path)
    x = point_cloud.x
    y = point_cloud.y
    z = point_cloud.z
    return np.vstack((x, y, z)).T

In [ ]:
wc = read_laz_file(r"Cloud.laz")

### Generate requied matrices for projection

In [ ]:
from projkit.camops import get_int_mat, get_R, get_t_from_R_C

In [ ]:
K = get_int_mat(fx, fy, Cx, Cy)
R = get_R(roll, pitch, yaw)
t = get_t_from_R_C(R, np.array(camera_center))
d = np.array(
            [
                k1,
                k2,
                P1,
                P2,
                k3,
            ]
        )
# print(f"K = {K}, \n\nR = {R}, \n\nt = {t}, \n\nd = {d}")

### Project Data

The function returns projected image coordinates 'ic', its corresponding world coordinates 'wc' and the z_values  

In [ ]:
from projkit.camops import project_in_2d_with_K_R_t_dist_coeff

In [ ]:
ic, wc, z_values = project_in_2d_with_K_R_t_dist_coeff(K, R, t, d, wc)

### Remove Data which does not lie in the image

In [ ]:
from projkit.imutils import to_image, filter_image_and_world_points_with_img_dim

In [ ]:
ic, wc, z = filter_image_and_world_points_with_img_dim(Nx, Ny, ic, wc)

In [ ]:
projectection_on_image = to_image(Ny, Nx, ic, wc)
plt.imshow(projectection_on_image, cmap='gray')